In [154]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [155]:
from datetime import datetime, timedelta

In [156]:
from math import ceil

In [157]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [158]:
import statsmodels.api as sm  
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings('ignore')

## Functions & Classes

In [159]:
def get_week_of_month(date):
   first_day = date.replace(day=1)

   day_of_month = date.day

   if(first_day.weekday() == 6):
       adjusted_dom = (1 + first_day.weekday()) / 7
   else:
       adjusted_dom = day_of_month + first_day.weekday()

   return int(ceil(adjusted_dom/7.0))

In [160]:
def reduce_memory_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df.loc[:,col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df.loc[:,col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df.loc[:,col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df.loc[:,col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df.loc[:,col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df.loc[:,col] = df[col].astype(np.float32)
                else:
                    df.loc[:,col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df  

In [161]:
def standard_column_names(df_pd):
      """
      Funtion to standardize column names Upper case the names, Replace spaces with underscore
      (,) with blanks,% with PER and - with undescore
      df: Input dataframe
      df_pd: Output with Standardzied column names
      """
      #df_pd=df.toPandas()
      df_pd.columns = df_pd.columns.str.strip().str.upper().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('%', 'PER').str.replace('-', '_')
      return df_pd

## Read data 

In [162]:
df_sales_price=pd.read_csv(r'C:\Users\kishor.kukreja\OneDrive - Fractal Analytics Pvt. Ltd\Desktop\WIP\Tech Challenge\Output\cleaned_sales_with_price.csv')

In [163]:
df_sales_price.shape

(240000, 25)

In [164]:
df_sales_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 25 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   product_id                   240000 non-null  object 
 1   channel                      240000 non-null  object 
 2   country                      240000 non-null  object 
 3   week_key                     240000 non-null  int64  
 4   current_price_x              240000 non-null  float64
 5   sales_volume                 240000 non-null  float64
 6   total_stock_volume           240000 non-null  float64
 7   store_stock_volume           240000 non-null  float64
 8   depot_stock_volume           240000 non-null  float64
 9   future_commitment_volume     240000 non-null  float64
 10  intake_volume                240000 non-null  float64
 11  per_stores_with_stock_count  240000 non-null  float64
 12  per_stores_with_sales_count  240000 non-null  float64
 13 

In [165]:
df_sales_price['key']=df_sales_price.apply(lambda x : x['product_id']+x['channel']+x['country'],axis=1)

In [166]:
df_xx=df_sales_price.groupby(['key','product_id', 'channel', 'country'])['sales_volume'].sum().reset_index()

In [167]:
remove_items=list(df_xx[df_xx['sales_volume']==0]['key'].unique())
df_filtered=df_sales_price[~df_sales_price['key'].isin(remove_items)]
df_filtered.shape

(111660, 26)

In [168]:
df_sales_price=df_filtered.copy()

In [169]:
df_sales_price=reduce_memory_usage(df_sales_price)

Mem. usage decreased to 13.52 Mb (41.2% reduction)


In [170]:
df_sales_price.head()

,product_id,channel,country,week_key,current_price_x,sales_volume,total_stock_volume,store_stock_volume,depot_stock_volume,future_commitment_volume,intake_volume,per_stores_with_stock_count,per_stores_with_sales_count,week,year,start_date,end_date,product_season,product_group,product_subgroup,product_class,product_subclass,original_price,previous_price,current_price_y,key
0,135fc45e,Stores,B,201502,18.984375,3.0,39.0,6.0,33.0,0.0,0.0,0.101013,0.007801,2,2015,2015-01-05,2015-01-11,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.28125,135fc45eStoresB
1,135fc45e,Stores,B,201503,18.984375,3.0,38.0,6.0,33.0,0.0,-1.0,0.101013,0.007801,3,2015,2015-01-12,2015-01-18,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.28125,135fc45eStoresB
2,135fc45e,Stores,B,201504,18.984375,3.0,38.0,6.0,33.0,0.0,-1.0,0.101013,0.007801,4,2015,2015-01-19,2015-01-25,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.28125,135fc45eStoresB
3,135fc45e,Stores,B,201505,18.984375,4.0,38.0,6.0,32.0,0.0,0.0,0.112671,0.011703,5,2015,2015-01-26,2015-02-01,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.28125,135fc45eStoresB
4,135fc45e,Stores,B,201506,18.984375,4.0,38.0,5.0,32.0,0.0,1.0,0.108826,0.011703,6,2015,2015-02-02,2015-02-08,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.28125,135fc45eStoresB


## Feature Engineer 

In [171]:
df_sales_price['start_date']=pd.to_datetime(df_sales_price['start_date'])

In [172]:
df_sales_price['end_date']=pd.to_datetime(df_sales_price['end_date'])

In [173]:
df_sales_price['week_sin365_1'] = np.sin(2*np.pi*df_sales_price['week']/52)
df_sales_price['week_cos365_1'] = np.cos(2*np.pi*df_sales_price['week']/52)
df_sales_price['week_sin365_2'] = np.sin(4*np.pi*df_sales_price['week']/52)
df_sales_price['week_cos365_2'] = np.cos(4*np.pi*df_sales_price['week']/52)

In [174]:
df_sales_price.head()

,product_id,channel,country,week_key,current_price_x,sales_volume,total_stock_volume,store_stock_volume,depot_stock_volume,future_commitment_volume,intake_volume,per_stores_with_stock_count,per_stores_with_sales_count,week,year,start_date,end_date,product_season,product_group,product_subgroup,product_class,product_subclass,original_price,previous_price,current_price_y,key,week_sin365_1,week_cos365_1,week_sin365_2,week_cos365_2
0,135fc45e,Stores,B,201502,18.984375,3.0,39.0,6.0,33.0,0.0,0.0,0.101013,0.007801,2,2015,2015-01-05,2015-01-11,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.28125,135fc45eStoresB,0.239316,0.970942,0.464723,0.885456
1,135fc45e,Stores,B,201503,18.984375,3.0,38.0,6.0,33.0,0.0,-1.0,0.101013,0.007801,3,2015,2015-01-12,2015-01-18,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.28125,135fc45eStoresB,0.354605,0.935016,0.663123,0.748511
2,135fc45e,Stores,B,201504,18.984375,3.0,38.0,6.0,33.0,0.0,-1.0,0.101013,0.007801,4,2015,2015-01-19,2015-01-25,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.28125,135fc45eStoresB,0.464723,0.885456,0.822984,0.568065
3,135fc45e,Stores,B,201505,18.984375,4.0,38.0,6.0,32.0,0.0,0.0,0.112671,0.011703,5,2015,2015-01-26,2015-02-01,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.28125,135fc45eStoresB,0.568065,0.822984,0.935016,0.354605
4,135fc45e,Stores,B,201506,18.984375,4.0,38.0,5.0,32.0,0.0,1.0,0.108826,0.011703,6,2015,2015-02-02,2015-02-08,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.28125,135fc45eStoresB,0.663123,0.748511,0.992709,0.120537


In [175]:
epsilon=1e-7

In [176]:
df_sales_price['price_change_wrt_previous']=df_sales_price.apply(lambda x: ((x['current_price_x']-x['previous_price'])/(x['previous_price']+epsilon)),axis=1)

In [177]:
df_sales_price['price_change_wrt_original']=df_sales_price.apply(lambda x: ((x['current_price_x']-x['original_price'])/(x['original_price']+epsilon)),axis=1)

In [178]:
df_sales_price.head(10)

,product_id,channel,country,week_key,current_price_x,sales_volume,total_stock_volume,store_stock_volume,depot_stock_volume,future_commitment_volume,intake_volume,per_stores_with_stock_count,per_stores_with_sales_count,week,year,start_date,end_date,product_season,product_group,product_subgroup,product_class,product_subclass,original_price,previous_price,current_price_y,key,week_sin365_1,week_cos365_1,week_sin365_2,week_cos365_2,price_change_wrt_previous,price_change_wrt_original
0,135fc45e,Stores,B,201502,18.984375,3.0,39.0,6.0,33.0,0.0,0.0,0.101013,0.007801,2,2015,2015-01-05,2015-01-11,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.281250,135fc45eStoresB,0.239316,0.970942,0.464723,0.885456,0.000000,0.000000
1,135fc45e,Stores,B,201503,18.984375,3.0,38.0,6.0,33.0,0.0,-1.0,0.101013,0.007801,3,2015,2015-01-12,2015-01-18,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.281250,135fc45eStoresB,0.354605,0.935016,0.663123,0.748511,0.000000,0.000000
2,135fc45e,Stores,B,201504,18.984375,3.0,38.0,6.0,33.0,0.0,-1.0,0.101013,0.007801,4,2015,2015-01-19,2015-01-25,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.281250,135fc45eStoresB,0.464723,0.885456,0.822984,0.568065,0.000000,0.000000
3,135fc45e,Stores,B,201505,18.984375,4.0,38.0,6.0,32.0,0.0,0.0,0.112671,0.011703,5,2015,2015-01-26,2015-02-01,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.281250,135fc45eStoresB,0.568065,0.822984,0.935016,0.354605,0.000000,0.000000
4,135fc45e,Stores,B,201506,18.984375,4.0,38.0,5.0,32.0,0.0,1.0,0.108826,0.011703,6,2015,2015-02-02,2015-02-08,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.281250,135fc45eStoresB,0.663123,0.748511,0.992709,0.120537,0.000000,0.000000
5,135fc45e,Stores,B,201507,16.281250,13.0,31.0,19.0,12.0,0.0,0.0,0.240967,0.042786,7,2015,2015-02-09,2015-02-15,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.281250,135fc45eStoresB,0.748511,0.663123,0.992709,-0.120537,-0.142387,-0.142387
6,135fc45e,Stores,B,201508,14.484375,26.0,34.0,22.0,11.0,0.0,1.0,0.256592,0.081604,8,2015,2015-02-16,2015-02-22,L,26387251,c3567a18,8face522,641d430c,18.984375,16.281250,14.484375,135fc45eStoresB,0.822984,0.568065,0.935016,-0.354605,-0.110365,-0.237037
7,135fc45e,Stores,B,201509,14.484375,28.0,31.0,20.0,11.0,0.0,0.0,0.264404,0.089417,9,2015,2015-02-23,2015-03-01,L,26387251,c3567a18,8face522,641d430c,18.984375,14.484375,13.132812,135fc45eStoresB,0.885456,0.464723,0.822984,-0.568065,0.000000,-0.237037
8,135fc45e,Stores,B,201510,14.484375,32.0,28.0,28.0,0.0,0.0,0.0,0.668457,0.120483,10,2015,2015-03-02,2015-03-08,L,26387251,c3567a18,8face522,641d430c,18.984375,14.484375,13.132812,135fc45eStoresB,0.935016,0.354605,0.663123,-0.748511,0.000000,-0.237037
9,135fc45e,Stores,B,201511,14.484375,34.0,25.0,25.0,0.0,0.0,0.0,0.583008,0.120483,11,2015,2015-03-09,2015-03-15,L,26387251,c3567a18,8face522,641d430c,18.984375,14.484375,13.132812,135fc45eStoresB,0.970942,0.239316,0.464723,-0.885456,0.000000,-0.237037


In [179]:
df_sales_price['week_of_month']=df_sales_price.apply(lambda x: get_week_of_month(x['start_date']),axis=1)

In [180]:
df_sales_price['is_month_start']=df_sales_price.apply(lambda x: 1 if x['week_of_month']==1 else 0,axis=1)

In [181]:
df_sales_price['is_month_end']=df_sales_price.apply(lambda x: 1 if x['week_of_month']>=4 else 0,axis=1)

In [182]:
df_sales_price['month']=df_sales_price['start_date'].dt.month

In [183]:
df_sales_price['is_year_start']=df_sales_price.apply(lambda x: 1 if x['month']>=2 else 0,axis=1)

In [184]:
df_sales_price['is_year_end']=df_sales_price.apply(lambda x: 1 if x['month']>=11 else 0,axis=1)

In [185]:
df_sales_price['is_quarter_start']=df_sales_price.apply(lambda x: 1 if (x['month']==1 or x['month']==4 or x['month']==7 or x['month']==10 ) and x['week_of_month']==1 else 0,axis=1)

In [186]:
df_sales_price['is_quarter_end']=df_sales_price.apply(lambda x: 1 if (x['month']==3 or x['month']==6 or x['month']==9 or x['month']==12 ) and x['week_of_month']>=1 else 0,axis=1)

In [187]:
df_sales_price.head()

,product_id,channel,country,week_key,current_price_x,sales_volume,total_stock_volume,store_stock_volume,depot_stock_volume,future_commitment_volume,intake_volume,per_stores_with_stock_count,per_stores_with_sales_count,week,year,start_date,end_date,product_season,product_group,product_subgroup,product_class,product_subclass,original_price,previous_price,current_price_y,key,week_sin365_1,week_cos365_1,week_sin365_2,week_cos365_2,price_change_wrt_previous,price_change_wrt_original,week_of_month,is_month_start,is_month_end,month,is_year_start,is_year_end,is_quarter_start,is_quarter_end
0,135fc45e,Stores,B,201502,18.984375,3.0,39.0,6.0,33.0,0.0,0.0,0.101013,0.007801,2,2015,2015-01-05,2015-01-11,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.28125,135fc45eStoresB,0.239316,0.970942,0.464723,0.885456,0.0,0.0,2,0,0,1,0,0,0,0
1,135fc45e,Stores,B,201503,18.984375,3.0,38.0,6.0,33.0,0.0,-1.0,0.101013,0.007801,3,2015,2015-01-12,2015-01-18,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.28125,135fc45eStoresB,0.354605,0.935016,0.663123,0.748511,0.0,0.0,3,0,0,1,0,0,0,0
2,135fc45e,Stores,B,201504,18.984375,3.0,38.0,6.0,33.0,0.0,-1.0,0.101013,0.007801,4,2015,2015-01-19,2015-01-25,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.28125,135fc45eStoresB,0.464723,0.885456,0.822984,0.568065,0.0,0.0,4,0,1,1,0,0,0,0
3,135fc45e,Stores,B,201505,18.984375,4.0,38.0,6.0,32.0,0.0,0.0,0.112671,0.011703,5,2015,2015-01-26,2015-02-01,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.28125,135fc45eStoresB,0.568065,0.822984,0.935016,0.354605,0.0,0.0,5,0,1,1,0,0,0,0
4,135fc45e,Stores,B,201506,18.984375,4.0,38.0,5.0,32.0,0.0,1.0,0.108826,0.011703,6,2015,2015-02-02,2015-02-08,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.28125,135fc45eStoresB,0.663123,0.748511,0.992709,0.120537,0.0,0.0,1,1,0,2,1,0,0,0


In [188]:
df_sales_price.columns

Index(['product_id', 'channel', 'country', 'week_key', 'current_price_x',
       'sales_volume', 'total_stock_volume', 'store_stock_volume',
       'depot_stock_volume', 'future_commitment_volume', 'intake_volume',
       'per_stores_with_stock_count', 'per_stores_with_sales_count', 'week',
       'year', 'start_date', 'end_date', 'product_season', 'product_group',
       'product_subgroup', 'product_class', 'product_subclass',
       'original_price', 'previous_price', 'current_price_y', 'key',
       'week_sin365_1', 'week_cos365_1', 'week_sin365_2', 'week_cos365_2',
       'price_change_wrt_previous', 'price_change_wrt_original',
       'week_of_month', 'is_month_start', 'is_month_end', 'month',
       'is_year_start', 'is_year_end', 'is_quarter_start', 'is_quarter_end'],
      dtype='object')

In [189]:
df_sales_price.shape

(111660, 40)

In [190]:
df_sales_price['year_2015']=np.where(df_sales_price['year']==2015,1,0)
df_sales_price['year_2016']=np.where(df_sales_price['year']==2016,1,0)

In [191]:
df_sales_price['month_sin365_1'] = np.sin(4*np.pi*df_sales_price['month']/12)
df_sales_price['month_cos365_1'] = np.cos(4*np.pi*df_sales_price['month']/12)

In [192]:
df_sales_price.head()

,product_id,channel,country,week_key,current_price_x,sales_volume,total_stock_volume,store_stock_volume,depot_stock_volume,future_commitment_volume,intake_volume,per_stores_with_stock_count,per_stores_with_sales_count,week,year,start_date,end_date,product_season,product_group,product_subgroup,product_class,product_subclass,original_price,previous_price,current_price_y,key,week_sin365_1,week_cos365_1,week_sin365_2,week_cos365_2,price_change_wrt_previous,price_change_wrt_original,week_of_month,is_month_start,is_month_end,month,is_year_start,is_year_end,is_quarter_start,is_quarter_end,year_2015,year_2016,month_sin365_1,month_cos365_1
0,135fc45e,Stores,B,201502,18.984375,3.0,39.0,6.0,33.0,0.0,0.0,0.101013,0.007801,2,2015,2015-01-05,2015-01-11,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.28125,135fc45eStoresB,0.239316,0.970942,0.464723,0.885456,0.0,0.0,2,0,0,1,0,0,0,0,1,0,0.866025,0.5
1,135fc45e,Stores,B,201503,18.984375,3.0,38.0,6.0,33.0,0.0,-1.0,0.101013,0.007801,3,2015,2015-01-12,2015-01-18,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.28125,135fc45eStoresB,0.354605,0.935016,0.663123,0.748511,0.0,0.0,3,0,0,1,0,0,0,0,1,0,0.866025,0.5
2,135fc45e,Stores,B,201504,18.984375,3.0,38.0,6.0,33.0,0.0,-1.0,0.101013,0.007801,4,2015,2015-01-19,2015-01-25,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.28125,135fc45eStoresB,0.464723,0.885456,0.822984,0.568065,0.0,0.0,4,0,1,1,0,0,0,0,1,0,0.866025,0.5
3,135fc45e,Stores,B,201505,18.984375,4.0,38.0,6.0,32.0,0.0,0.0,0.112671,0.011703,5,2015,2015-01-26,2015-02-01,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.28125,135fc45eStoresB,0.568065,0.822984,0.935016,0.354605,0.0,0.0,5,0,1,1,0,0,0,0,1,0,0.866025,0.5
4,135fc45e,Stores,B,201506,18.984375,4.0,38.0,5.0,32.0,0.0,1.0,0.108826,0.011703,6,2015,2015-02-02,2015-02-08,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.28125,135fc45eStoresB,0.663123,0.748511,0.992709,0.120537,0.0,0.0,1,1,0,2,1,0,0,0,1,0,0.866025,-0.5


In [193]:
df_sales_price.columns

Index(['product_id', 'channel', 'country', 'week_key', 'current_price_x',
       'sales_volume', 'total_stock_volume', 'store_stock_volume',
       'depot_stock_volume', 'future_commitment_volume', 'intake_volume',
       'per_stores_with_stock_count', 'per_stores_with_sales_count', 'week',
       'year', 'start_date', 'end_date', 'product_season', 'product_group',
       'product_subgroup', 'product_class', 'product_subclass',
       'original_price', 'previous_price', 'current_price_y', 'key',
       'week_sin365_1', 'week_cos365_1', 'week_sin365_2', 'week_cos365_2',
       'price_change_wrt_previous', 'price_change_wrt_original',
       'week_of_month', 'is_month_start', 'is_month_end', 'month',
       'is_year_start', 'is_year_end', 'is_quarter_start', 'is_quarter_end',
       'year_2015', 'year_2016', 'month_sin365_1', 'month_cos365_1'],
      dtype='object')

In [194]:
df_sales_price['percent_change']=(df_sales_price['original_price']-df_sales_price['current_price_x'])/(df_sales_price['original_price'])

In [195]:
df_sales_price['min_percent']=np.where(df_sales_price['percent_change']>0.6,0.4,(np.where(df_sales_price['percent_change']>0.4,0.33,0.25)))

In [196]:
df_sales_price['max_cost']=df_sales_price['current_price_x']*(1-df_sales_price['min_percent'])

In [197]:
df_sales_price['min_cost']=df_sales_price['current_price_x']*(1-0.5)

In [198]:
df_sales_price[df_sales_price['max_cost']<=df_sales_price['min_cost']]

,product_id,channel,country,week_key,current_price_x,sales_volume,total_stock_volume,store_stock_volume,depot_stock_volume,future_commitment_volume,intake_volume,per_stores_with_stock_count,per_stores_with_sales_count,week,year,start_date,end_date,product_season,product_group,product_subgroup,product_class,product_subclass,original_price,previous_price,current_price_y,key,week_sin365_1,week_cos365_1,week_sin365_2,week_cos365_2,price_change_wrt_previous,price_change_wrt_original,week_of_month,is_month_start,is_month_end,month,is_year_start,is_year_end,is_quarter_start,is_quarter_end,year_2015,year_2016,month_sin365_1,month_cos365_1,percent_change,min_percent,max_cost,min_cost


In [199]:
df_sales_price[df_sales_price['current_price_x']<=df_sales_price['max_cost']]

,product_id,channel,country,week_key,current_price_x,sales_volume,total_stock_volume,store_stock_volume,depot_stock_volume,future_commitment_volume,intake_volume,per_stores_with_stock_count,per_stores_with_sales_count,week,year,start_date,end_date,product_season,product_group,product_subgroup,product_class,product_subclass,original_price,previous_price,current_price_y,key,week_sin365_1,week_cos365_1,week_sin365_2,week_cos365_2,price_change_wrt_previous,price_change_wrt_original,week_of_month,is_month_start,is_month_end,month,is_year_start,is_year_end,is_quarter_start,is_quarter_end,year_2015,year_2016,month_sin365_1,month_cos365_1,percent_change,min_percent,max_cost,min_cost


In [200]:
df_sales_price.to_csv(r'C:\Users\kishor.kukreja\OneDrive - Fractal Analytics Pvt. Ltd\Desktop\WIP\Tech Challenge\Output\data_with_feature_engineering.csv',index=False)

## Fit Linear model to find volume-price equation

In [134]:
df_sales_price.head()

,product_id,channel,country,week_key,current_price_x,sales_volume,total_stock_volume,store_stock_volume,depot_stock_volume,future_commitment_volume,intake_volume,per_stores_with_stock_count,per_stores_with_sales_count,week,year,start_date,end_date,product_season,product_group,product_subgroup,product_class,product_subclass,original_price,previous_price,current_price_y,key,week_sin365_1,week_cos365_1,price_change_wrt_previous,price_change_wrt_original,week_of_month,is_month_start,is_month_end,month,is_year_start,is_year_end,is_quarter_start,is_quarter_end,year_2015,year_2016,month_sin365_1,month_cos365_1,percent_change,min_percent,max_cost,min_cost
0,135fc45e,Stores,B,201502,18.984375,3.0,39.0,6.0,33.0,0.0,0.0,0.101013,0.007801,2,2015,2015-01-05,2015-01-11,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.28125,135fc45eStoresB,0.239316,0.970942,0.0,0.0,2,0,0,1,0,0,0,0,1,0,0.866025,0.5,0.0,0.25,14.238281,9.492188
1,135fc45e,Stores,B,201503,18.984375,3.0,38.0,6.0,33.0,0.0,-1.0,0.101013,0.007801,3,2015,2015-01-12,2015-01-18,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.28125,135fc45eStoresB,0.354605,0.935016,0.0,0.0,3,0,0,1,0,0,0,0,1,0,0.866025,0.5,0.0,0.25,14.238281,9.492188
2,135fc45e,Stores,B,201504,18.984375,3.0,38.0,6.0,33.0,0.0,-1.0,0.101013,0.007801,4,2015,2015-01-19,2015-01-25,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.28125,135fc45eStoresB,0.464723,0.885456,0.0,0.0,4,0,1,1,0,0,0,0,1,0,0.866025,0.5,0.0,0.25,14.238281,9.492188
3,135fc45e,Stores,B,201505,18.984375,4.0,38.0,6.0,32.0,0.0,0.0,0.112671,0.011703,5,2015,2015-01-26,2015-02-01,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.28125,135fc45eStoresB,0.568065,0.822984,0.0,0.0,5,0,1,1,0,0,0,0,1,0,0.866025,0.5,0.0,0.25,14.238281,9.492188
4,135fc45e,Stores,B,201506,18.984375,4.0,38.0,5.0,32.0,0.0,1.0,0.108826,0.011703,6,2015,2015-02-02,2015-02-08,L,26387251,c3567a18,8face522,641d430c,18.984375,18.984375,16.28125,135fc45eStoresB,0.663123,0.748511,0.0,0.0,1,1,0,2,1,0,0,0,1,0,0.866025,-0.5,0.0,0.25,14.238281,9.492188


In [135]:
df_sales_price=standard_column_names(df_sales_price)

In [136]:
df_sales_price.columns

Index(['PRODUCT_ID', 'CHANNEL', 'COUNTRY', 'WEEK_KEY', 'CURRENT_PRICE_X',
       'SALES_VOLUME', 'TOTAL_STOCK_VOLUME', 'STORE_STOCK_VOLUME',
       'DEPOT_STOCK_VOLUME', 'FUTURE_COMMITMENT_VOLUME', 'INTAKE_VOLUME',
       'PER_STORES_WITH_STOCK_COUNT', 'PER_STORES_WITH_SALES_COUNT', 'WEEK',
       'YEAR', 'START_DATE', 'END_DATE', 'PRODUCT_SEASON', 'PRODUCT_GROUP',
       'PRODUCT_SUBGROUP', 'PRODUCT_CLASS', 'PRODUCT_SUBCLASS',
       'ORIGINAL_PRICE', 'PREVIOUS_PRICE', 'CURRENT_PRICE_Y', 'KEY',
       'WEEK_SIN365_1', 'WEEK_COS365_1', 'PRICE_CHANGE_WRT_PREVIOUS',
       'PRICE_CHANGE_WRT_ORIGINAL', 'WEEK_OF_MONTH', 'IS_MONTH_START',
       'IS_MONTH_END', 'MONTH', 'IS_YEAR_START', 'IS_YEAR_END',
       'IS_QUARTER_START', 'IS_QUARTER_END', 'YEAR_2015', 'YEAR_2016',
       'MONTH_SIN365_1', 'MONTH_COS365_1', 'PERCENT_CHANGE', 'MIN_PERCENT',
       'MAX_COST', 'MIN_COST'],
      dtype='object')

In [137]:
df_vol_price=df_sales_price[[
       'KEY',
       'PRODUCT_ID', 'CHANNEL', 'COUNTRY',  'CURRENT_PRICE_X',
       'SALES_VOLUME', 
       'WEEK','WEEK_OF_MONTH','MONTH', 
       'ORIGINAL_PRICE', 'PREVIOUS_PRICE', 
       'WEEK_SIN365_1','WEEK_COS365_1', 
       'MONTH_SIN365_1', 'MONTH_COS365_1',
       'MAX_COST', 'MIN_COST',
       #'PRICE_CHANGE_WRT_PREVIOUS', 'PRICE_CHANGE_WRT_ORIGINAL',
       'IS_MONTH_START', 'IS_MONTH_END', 
       'IS_YEAR_START', 'IS_YEAR_END', 'IS_QUARTER_START', 'IS_QUARTER_END','YEAR_2015', 'YEAR_2016']]

In [138]:
df_vol_price.rename(columns={'CURRENT_PRICE_X':'CURRENT_PRICE'},inplace=True)

In [139]:
df_vol_price.head()

,KEY,PRODUCT_ID,CHANNEL,COUNTRY,CURRENT_PRICE,SALES_VOLUME,WEEK,WEEK_OF_MONTH,MONTH,ORIGINAL_PRICE,PREVIOUS_PRICE,WEEK_SIN365_1,WEEK_COS365_1,MONTH_SIN365_1,MONTH_COS365_1,MAX_COST,MIN_COST,IS_MONTH_START,IS_MONTH_END,IS_YEAR_START,IS_YEAR_END,IS_QUARTER_START,IS_QUARTER_END,YEAR_2015,YEAR_2016
0,135fc45eStoresB,135fc45e,Stores,B,18.984375,3.0,2,2,1,18.984375,18.984375,0.239316,0.970942,0.866025,0.5,14.238281,9.492188,0,0,0,0,0,0,1,0
1,135fc45eStoresB,135fc45e,Stores,B,18.984375,3.0,3,3,1,18.984375,18.984375,0.354605,0.935016,0.866025,0.5,14.238281,9.492188,0,0,0,0,0,0,1,0
2,135fc45eStoresB,135fc45e,Stores,B,18.984375,3.0,4,4,1,18.984375,18.984375,0.464723,0.885456,0.866025,0.5,14.238281,9.492188,0,1,0,0,0,0,1,0
3,135fc45eStoresB,135fc45e,Stores,B,18.984375,4.0,5,5,1,18.984375,18.984375,0.568065,0.822984,0.866025,0.5,14.238281,9.492188,0,1,0,0,0,0,1,0
4,135fc45eStoresB,135fc45e,Stores,B,18.984375,4.0,6,1,2,18.984375,18.984375,0.663123,0.748511,0.866025,-0.5,14.238281,9.492188,1,0,1,0,0,0,1,0


In [140]:
df_vol_price['channel_stores']=np.where(df_vol_price['CHANNEL']=='Stores',1,0)
df_vol_price['channel_online']=np.where(df_vol_price['CHANNEL']=='Online',1,0)

In [141]:
df_vol_price['country_A']=np.where(df_vol_price['COUNTRY']=='A',1,0)
df_vol_price['country_B']=np.where(df_vol_price['COUNTRY']=='B',1,0)

In [142]:
df_vol_price=standard_column_names(df_vol_price)
df_vol_price.columns

Index(['KEY', 'PRODUCT_ID', 'CHANNEL', 'COUNTRY', 'CURRENT_PRICE',
       'SALES_VOLUME', 'WEEK', 'WEEK_OF_MONTH', 'MONTH', 'ORIGINAL_PRICE',
       'PREVIOUS_PRICE', 'WEEK_SIN365_1', 'WEEK_COS365_1', 'MONTH_SIN365_1',
       'MONTH_COS365_1', 'MAX_COST', 'MIN_COST', 'IS_MONTH_START',
       'IS_MONTH_END', 'IS_YEAR_START', 'IS_YEAR_END', 'IS_QUARTER_START',
       'IS_QUARTER_END', 'YEAR_2015', 'YEAR_2016', 'CHANNEL_STORES',
       'CHANNEL_ONLINE', 'COUNTRY_A', 'COUNTRY_B'],
      dtype='object')

In [143]:
#Linear Regression using best features ,and obtaining R2s
#obtaining coeffs 
import statsmodels.api as sm  
from sklearn.preprocessing import PolynomialFeatures
polynomial_features= PolynomialFeatures(degree=3)

In [211]:
df_coeffecients = pd.DataFrame()

intercept_l =[]
current_price_l=[]

week_l=[]
week_month_l=[]
month_l=[]
year_2015_l=[]
year_2016_l=[]

original_price_l=[]
previous_price_l=[]

week_sin365_1_l=[]
week_cos365_1_l=[]
month_sin365_1_l=[]
month_cos365_1_l=[]

max_cost_l=[]

channel_online_l=[]
channel_stores_l=[]

country_A_l=[]
country_B_l=[]

#price_change_previous_l=[]
#price_change_original_l=[]


is_month_start_l=[]
is_month_end_l=[]
is_year_start_l=[]
is_year_end_l=[]
is_quarter_start_l=[]
is_quarter_end_l=[]



product_l=[]
channel_l=[]
country_l=[]
key_l=[]


In [212]:
product_list=df_vol_price.PRODUCT_ID.unique()
channel_list=df_vol_price.CHANNEL.unique()
country_list=df_vol_price.COUNTRY.unique()

In [213]:
r_squared=[]


In [214]:
for country in country_list:
    for channel in channel_list:
        for product in product_list:
            
            df_x=df_vol_price[(df_vol_price.PRODUCT_ID==product)&(df_vol_price.CHANNEL==channel)&(df_vol_price.COUNTRY==country)]
            #df_x['INTERCEPT']=1
            
            if df_x.shape[0]!=0:
            
                key=product+channel+country
                #print(key)
                #print(df_x)
                y = df_x['SALES_VOLUME']
                X = df_x[['CURRENT_PRICE',
                           'WEEK','WEEK_OF_MONTH','MONTH', 
                           'YEAR_2015','YEAR_2016', 
                           'ORIGINAL_PRICE', 'PREVIOUS_PRICE', 
                           'WEEK_SIN365_1','WEEK_COS365_1', 
                           'MONTH_SIN365_1','MONTH_COS365_1', 
                           #'PRICE_CHANGE_WRT_PREVIOUS', 'PRICE_CHANGE_WRT_ORIGINAL',
                           'IS_MONTH_START', 'IS_MONTH_END', 
                           'IS_YEAR_START', 'IS_YEAR_END', 'IS_QUARTER_START', 'IS_QUARTER_END',
                            'MAX_COST',
                            'COUNTRY_A','COUNTRY_B','CHANNEL_STORES','CHANNEL_ONLINE'
                         
                         ]]

                #X = sm.add_constant(X)

                #X = np.append(arr= np.ones((df_x.shape[0], 1)).astype(int), values=X, axis=1)

                #lm = sm.OLS(y,X).fit()

                #lm=sm.OLS(formula = df_x.SALES_VOLUME + ' ~ ' + ' + '.join(df_x.columns.tolist()), data = df_x).fit()

                lm=sm.OLS.from_formula(formula = 'SALES_VOLUME ~ CURRENT_PRICE+WEEK+WEEK_OF_MONTH+MONTH+YEAR_2015+YEAR_2016+ORIGINAL_PRICE+PREVIOUS_PRICE+WEEK_SIN365_1+WEEK_COS365_1+MONTH_SIN365_1+MONTH_COS365_1+IS_MONTH_START+IS_MONTH_END+IS_YEAR_START+IS_YEAR_END+IS_QUARTER_START+IS_QUARTER_END + MAX_COST+COUNTRY_A+COUNTRY_B+CHANNEL_STORES+CHANNEL_ONLINE' 
     , data = df_x).fit()

                #y_predict = lm.predict(X)

                A = pd.read_html(lm.summary().tables[1].as_html(),header=0,index_col=0)[0]
                B = pd.read_html(lm.summary().tables[0].as_html(),header=0)[0]
                #print(B)
                #print()
                
                r_squared.append(B.loc[0][3])

                #print(lm.summary())

                #print(lm)

                const = A['coef'].values[0]
                CURRENT_PRICE = A['coef'].values[1]
                WEEK = A['coef'].values[2]
                WEEK_OF_MONTH = A['coef'].values[3]
                MONTH = A['coef'].values[4]
                YEAR_2015 = A['coef'].values[5]
                YEAR_2016 = A['coef'].values[6]

                ORIGINAL_PRICE = A['coef'].values[7]
                PREVIOUS_PRICE = A['coef'].values[8]

                WEEK_SIN365_1 = A['coef'].values[9]
                WEEK_COS365_1 = A['coef'].values[10]
                MONTH_SIN365_1 = A['coef'].values[11]
                MONTH_COS365_1 = A['coef'].values[12]

                #PRICE_CHANGE_WRT_PREVIOUS = A['coef'].values[12]
                #PRICE_CHANGE_WRT_ORIGINAL = A['coef'].values[12]
                IS_MONTH_START = A['coef'].values[13]
                IS_MONTH_END = A['coef'].values[14]

                IS_YEAR_START = A['coef'].values[15]
                IS_YEAR_END = A['coef'].values[16]

                IS_QUARTER_START = A['coef'].values[17]
                IS_QUARTER_END = A['coef'].values[18]
                
                
                MAX_COST = A['coef'].values[19]

                COUNTRY_A = A['coef'].values[20]
                COUNTRY_B = A['coef'].values[21]

                CHANNEL_STORES = A['coef'].values[22]
                CHANNEL_ONLINE = A['coef'].values[23]


                #print(lm.params)

                key_l.append(key)
                product_l.append(product)
                channel_l.append(channel)
                country_l.append(country)

                intercept_l.append(const)

                current_price_l.append(CURRENT_PRICE)
                week_l.append(WEEK)
                week_month_l.append(WEEK_OF_MONTH)
                month_l.append(MONTH)
                year_2015_l.append(YEAR_2015)
                year_2016_l.append(YEAR_2016)

                original_price_l.append(ORIGINAL_PRICE)
                previous_price_l.append(PREVIOUS_PRICE)

                week_sin365_1_l.append(WEEK_SIN365_1)
                week_cos365_1_l.append(WEEK_COS365_1)
                month_sin365_1_l.append(MONTH_SIN365_1)
                month_cos365_1_l.append(MONTH_COS365_1)

                #price_change_previous_l.append(PRICE_CHANGE_WRT_PREVIOUS)
                #price_change_original_l.append(PRICE_CHANGE_WRT_ORIGINAL)


                is_month_start_l.append(IS_MONTH_START)
                is_month_end_l.append(IS_MONTH_END)
                is_year_start_l.append(IS_YEAR_START)
                is_year_end_l.append(IS_YEAR_END)
                is_quarter_start_l.append(IS_QUARTER_START)
                is_quarter_end_l.append(IS_QUARTER_END)
                
                
                max_cost_l.append(MAX_COST)
                country_A_l.append(COUNTRY_A)
                country_B_l.append(COUNTRY_B)
                channel_stores_l.append(CHANNEL_STORES)
                channel_online_l.append(CHANNEL_ONLINE)
            

In [216]:
df_coeffecients=pd.DataFrame()
df_coeffecients['PRODUCT_ID']=product_l
df_coeffecients['CHANNEL']=channel_l
df_coeffecients['COUNTRY']=country_l
df_coeffecients['KEY']=key_l
df_coeffecients['INTERCEPT']=intercept_l

df_coeffecients['CURRENT_PRICE_COEFF']=current_price_l
df_coeffecients['WEEK_COEFF']=week_l
df_coeffecients['MONTH_COEFF']=week_month_l
df_coeffecients['WEEK_OF_MONTH_COEFF']=month_l
df_coeffecients['YEAR_2015_COEFF']=year_2015_l
df_coeffecients['YEAR_2016_COEFF']=year_2016_l


df_coeffecients['ORIGINAL_PRICE_COEFF']=original_price_l
df_coeffecients['PREVIOUS_PRICE_COEFF']=previous_price_l


df_coeffecients['WEEK_SIN365_1_COEFF']=week_sin365_1_l
df_coeffecients['WEEK_COS365_1_COEFF']=week_cos365_1_l
df_coeffecients['MONTH_SIN365_1_COEFF']=month_sin365_1_l
df_coeffecients['MONTH_COS365_1_COEFF']=month_cos365_1_l


#df_coeffecients['PRICE_CHANGE_WRT_PREVIOUS']=price_change_previous_l
#df_coeffecients['PRICE_CHANGE_WRT_ORIGINAL']=price_change_original_l


df_coeffecients['IS_MONTH_START_COEFF']=is_month_start_l
df_coeffecients['IS_MONTH_END_COEFF']=is_month_end_l
df_coeffecients['IS_YEAR_START_COEFF']=is_year_start_l
df_coeffecients['IS_YEAR_END_COEFF']=is_year_end_l
df_coeffecients['IS_QUARTER_START_COEFF']=is_quarter_start_l
df_coeffecients['IS_QUARTER_END_COEFF']=is_quarter_end_l

df_coeffecients['MAX_COST_COEFF']=max_cost_l
df_coeffecients['COUNTRY_A_COEFF']=country_A_l
df_coeffecients['COUNTRY_B_COEFF']=country_B_l
df_coeffecients['CHANNEL_STORES_COEFF']=channel_stores_l
df_coeffecients['CHANNEL_ONLINE_COEFF']=channel_online_l
#df_coeffecients['R_SQUARED']=r_squared

In [217]:
df_coeffecients.shape

(1861, 29)

In [218]:
df_coeffecients.columns

Index(['PRODUCT_ID', 'CHANNEL', 'COUNTRY', 'KEY', 'INTERCEPT',
       'CURRENT_PRICE_COEFF', 'WEEK_COEFF', 'MONTH_COEFF',
       'WEEK_OF_MONTH_COEFF', 'YEAR_2015_COEFF', 'YEAR_2016_COEFF',
       'ORIGINAL_PRICE_COEFF', 'PREVIOUS_PRICE_COEFF', 'WEEK_SIN365_1_COEFF',
       'WEEK_COS365_1_COEFF', 'MONTH_SIN365_1_COEFF', 'MONTH_COS365_1_COEFF',
       'IS_MONTH_START_COEFF', 'IS_MONTH_END_COEFF', 'IS_YEAR_START_COEFF',
       'IS_YEAR_END_COEFF', 'IS_QUARTER_START_COEFF', 'IS_QUARTER_END_COEFF',
       'MAX_COST_COEFF', 'COUNTRY_A_COEFF', 'COUNTRY_B_COEFF',
       'CHANNEL_STORES_COEFF', 'CHANNEL_ONLINE_COEFF', 'R_SQUARED'],
      dtype='object')

In [221]:
#df_coeffecients[df_coeffecients['R_SQUARED']>=0.8].shape

(578, 29)

In [150]:
#saving the dataset
df_coeffecients.to_csv(r'C:\Users\kishor.kukreja\OneDrive - Fractal Analytics Pvt. Ltd\Desktop\WIP\Tech Challenge\Model Output\Sales Volume - Price Coeffecients.csv',index=False)

In [151]:
df_coeffecients.KEY.nunique()

1861

In [152]:
df_coeffecients.CHANNEL.nunique()

2

In [153]:
df_coeffecients.COUNTRY.nunique()

2